<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci553-data-mining-sp24/blob/main/assignment-3/notebooks/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Installation

In [1]:
!pip install pyspark ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2d72f3b76bf782b4e89c08cdd04eee275c990db92b53961fcaaf5ee219d52e2c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
%%bash
java --version

openjdk 11.0.22 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)


# Imports

In [3]:
import os
import sys
import json
import pandas as pd
from itertools import combinations
import math
import statistics
from pyspark import SparkContext
import numpy as np

from pprint import pprint

%load_ext autotime

time: 289 µs (started: 2024-03-17 08:31:04 +00:00)


In [4]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/DSCI553/hw3")
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/DSCI553/hw3'

time: 214 ms (started: 2024-03-17 08:31:04 +00:00)


In [5]:
!ls

HW3.ipynb	t1.csv	    t2_1.csv  t2_2-dryrun.csv  task1_ref.py  task2_1_ref.py
HW3StudentData	t1-ref.txt  t2_2.csv  t2-ref.txt       task2_1.py    task2_2.py
time: 406 ms (started: 2024-03-17 08:31:05 +00:00)


# Tasks

## 1

In [ ]:
import sys
import time
from pyspark import SparkContext, SparkConf
from itertools import combinations
import random
import csv


random.seed(37)

# Define constants
NUM_HASH_FUNCTIONS = 50
PRIME_NUMBER = 15485863
ROWS_PER_BAND = 2
BANDS = NUM_HASH_FUNCTIONS // ROWS_PER_BAND


def prepare_dataset(data):
    # Remove the header
    header = data.first()
    data = (
        data.filter(lambda row: row != header)
        .map(lambda row: row.split(","))
    )

    # Find unique users and map it to an index
    usr_to_idx = (
        data.map(lambda x: x[0])
        .distinct()
        .zipWithIndex()
        .collectAsMap()
    )

    # Group users that has reviewed a business
    business_user = (
        data.map(lambda row: (row[1], [row[0]]))
        .reduceByKey(lambda a, b: a + b)
    )
    return business_user, usr_to_idx


def generate_hash_function_params(max_range, count):
    """Generate random hash function parameters within a specified range."""
    hash_funcs = []
    for _ in range(count):
        a = random.randint(1, max_range)  # Random coefficient 'a'
        b = random.randint(0, max_range)  # Random intercept 'b'
        hash_funcs.append((a, b))
    return hash_funcs


def hash_item(item, params, num_bins):
    """Hash an item using given hash function parameters.
    Calculate hash value using the formula: ((a * item + b) % PRIME_NUMBER) % num_bins
    """
    hash_val = ((params[0] * item + params[1]) % PRIME_NUMBER) % num_bins
    return hash_val


def build_minhash_signature_matrix(hash_funcs, users, num_bins):
    """Build the minhash signature matrix for a set of users."""
    mhs = []
    for params in hash_funcs:
        minhash = float("inf")
        for user in users:
            # Hash each user and find the minimum hash value
            hash_val = hash_item(user, params, num_bins)
            minhash = min(minhash, hash_val)
        mhs.append(minhash)
    return mhs


def jaccard_similarity(pair, bus_user_dict):
    """
    Calculate Jaccard similarity for a candidate pair of businesses.

    Args:
        pair (tuple): A pair of business IDs.
        bus_user_dict (dict): Dictionary mapping business IDs to sets of user IDs.

    Returns:
        tuple: A tuple containing the business pair and their Jaccard similarity.
    """
    # Extract business IDs from the pair
    bus1, bus2 = pair

    # Get sets of users who reviewed each business
    user1 = set(bus_user_dict[bus1])
    user2 = set(bus_user_dict[bus2])

    # Calculate Jaccard similarity
    intersection = len(user1 & user2)
    union = len(user1 | user2)
    similarity = intersection / union if union != 0 else 0

    return (bus1, bus2), similarity


def jaccard_based_lsh(prepared_data):
    """Perform Jaccard-based Locality Sensitive Hashing (LSH) on prepared data.

    This function applies LSH to find candidate pairs of businesses with similar users,
    based on the Jaccard similarity metric.

    Algorithm Steps:
    1. Unpack the prepared data containing the business-to-user mapping and user index mapping.
    2. Generate a set of hash functions.
    3. Compute the Minhash Signature for each business.
    4. Divide the signature matrix into bands.
    5. Group businesses into bands based on their Minhash Signature.
    6. Find candidate pairs of businesses within each band.
    7. Calculate the Jaccard similarity for candidate pairs.
    8. Filter pairs with similarity above a threshold (e.g., 0.5).
    9. Sort the results by business ID pairs.
    10. Return the RDD containing the Jaccard similarity results for candidate business pairs.

    Args:
        prepared_data (tuple): A tuple containing the business-to-user mapping RDD and user index mapping dictionary.

    Returns:
        RDD: An RDD containing the Jaccard similarity results for candidate business pairs.
    """
    # Unpack prepared data
    business_to_user, usr_to_idx = prepared_data

    # Generate Hash functions
    NUM_BINS = len(usr_to_idx)
    hash_func_params = generate_hash_function_params(NUM_BINS, NUM_HASH_FUNCTIONS)

    # Compute Minhash Signature
    minhash_sign = (
        business_to_user.mapValues(lambda users: [usr_to_idx[user] for user in users])
        .mapValues(lambda users: build_minhash_signature_matrix(hash_func_params, users, NUM_BINS))
    )

    # Divide signature matrix into bands
    bands = (
        minhash_sign.flatMap(
            lambda x: [
                (
                    (i, tuple(x[1][i*ROWS_PER_BAND: (i+1)*ROWS_PER_BAND])), x[0]
                )
                for i in range(BANDS)
            ]
        )
        .groupByKey()
        .mapValues(list)
        .filter(lambda x: len(x[1]) > 1)
    )

    # Find the business candidate pairs
    candidates = (
        bands.map(lambda x: sorted(x[1]))
        .flatMap(lambda x: list(combinations(x, 2)))
        .distinct()
    )

    # Calculate Jaccard Similirality for pairs
    bus_to_user_dict = business_to_user.collectAsMap()

    jaccard_sim_results = (
        candidates.map(lambda x: jaccard_similarity(x, bus_to_user_dict))
        .filter(lambda x: x[1] >= 0.5)
        .sortByKey()
        .map(lambda x: [x[0][0], x[0][1] ,x[1]])
    )
    return jaccard_sim_results


def task1(input_file_name, output_file_name):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 1")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        start_time = time.time()

        # Read the input data
        data = spark.textFile(input_file_name)
        prepared_data = prepare_dataset(data)

        # Compute Jaccard similarity using LSH
        jaccard_sim_results = jaccard_based_lsh(prepared_data)

        # Write header and results to a CSV file
        header = ["business_id_1", "business_id_2", "similarity"]
        with open(output_file_name, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerows(jaccard_sim_results.collect())

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     if len(sys.argv) != 3:
#         print(
#             "Usage: spark-submit task1.py <input_file_name> <output_file_name>"
#         )
#         sys.exit(1)

#     # Read input parameters
#     input_file_path = sys.argv[1]
#     output_file_path = sys.argv[2]

#     task1(input_file_path, output_file_path)


task1("HW3StudentData/yelp_train.csv", "t1.csv")

Duration: 53.42814826965332

time: 54.7 s (started: 2024-03-13 09:12:29 +00:00)


### Ref

In [ ]:
# @title Reference Code Task 1 { vertical-output: true, form-width: "30%" }
%%writefile task1_ref.py
from pyspark import SparkContext
import sys
import time
import random
from itertools import combinations
import operator

if __name__ == '__main__':
    input_path = sys.argv[1]
    output_path = sys.argv[2]

    s_t = time.time()

    spark = SparkContext(appName= "task1")
    lines = spark.textFile(input_path)
    first = lines.first()
    lines = lines.filter(lambda row: row != first).map(lambda row: row.split(","))
    #print(raw_rdd.take(10))

    bus_user = lines.map(lambda row: (row[1], row[0])).groupByKey().mapValues(set)
    #print(bus_user.take(10))
    bus_user_dict = {}
    for bus, users in bus_user.collect():
        bus_user_dict[bus] = users
    users = lines.map(lambda row: row[0]).distinct()
    users_dict = {}
    i = 0
    for user in users.collect():
        users_dict[user] = i
        i += 1

    n = 60
    m = i
    p = 1e9 + 7
    hash_funcs = [] #[a, b]
    a = random.sample(range(1, m), n)
    hash_funcs.append(a)
    b = random.sample(range(1, m), n)
    hash_funcs.append(b)
    #print(hash_funcs)

    sign_dict = {}
    for bus, user_list in bus_user.collect():
        minhash_sign_list = []
        for i in range(n):
            minhash = float("inf")
            for user in user_list:
                minhash = min(minhash, (((hash_funcs[0][i] * users_dict[user] + hash_funcs[1][i]) % p) % m))
            minhash_sign_list.append(int(minhash))
        sign_dict[bus] = minhash_sign_list
    #print(sign_dict)

    r = 2
    b = n // r
    bands_dict = {}
    for bus, minhash_sign in sign_dict.items():
        for i in range(0, b):
            #print(s[1][i*r: i*r+r])
            idx = (i, tuple(minhash_sign[i*r: i*r+r]))
            if idx not in bands_dict.keys():
                   bands_dict[idx] = []
                   bands_dict[idx].append(bus)
            else:
                   bands_dict[idx].append(bus)
    #print(bands_dict)
    bands_dict_fi = {}
    for key, values in bands_dict.items():
        if len(values) > 1:
            bands_dict_fi[key] = values
    #print(bands_dict_fi)
    #418426
    candidates = set()
    for values in bands_dict_fi.values():
        comb_list = combinations(sorted(values), 2)
        for item in comb_list:
            candidates.add(item)
    #print(candidates)

    result = {}
    for bus1, bus2 in candidates:
        user1 = bus_user_dict[bus1]
        user2 = bus_user_dict[bus2]
        js = len(user1 & user2) / len(user1 | user2)
        if js >= 0.5:
            result[str(bus1) + "," + str(bus2)] = js
    result = dict(sorted(result.items(), key=operator.itemgetter(0)))
    result_str = "business_id_1, business_id_2, similarity\n"
    for key, values in result.items():
        result_str += key + "," + str(values) + "\n"
    with open(output_path, "w") as f:
        f.writelines(result_str)

    e_t = time.time()
    print('Duration: ', e_t - s_t)

Overwriting task1_ref.py
time: 13.6 ms (started: 2024-03-13 03:07:25 +00:00)


In [ ]:
!spark-submit task1_ref.py HW3StudentData/yelp_train.csv t1-ref.txt --executor-memory 4G --driver-memory 4G

24/03/13 03:07:40 INFO SparkContext: Running Spark version 3.5.1
24/03/13 03:07:40 INFO SparkContext: OS info Linux, 6.1.58+, amd64
24/03/13 03:07:40 INFO SparkContext: Java version 11.0.22
24/03/13 03:07:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/13 03:07:40 INFO ResourceUtils: ==============================================================
24/03/13 03:07:40 INFO ResourceUtils: No custom resources configured for spark.driver.
24/03/13 03:07:40 INFO ResourceUtils: ==============================================================
24/03/13 03:07:40 INFO SparkContext: Submitted application: task1
24/03/13 03:07:41 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount

### Test

In [ ]:
((7 * 9098 + 147) % 15485863) % 50

33

time: 5.36 ms (started: 2024-03-13 00:29:40 +00:00)


In [ ]:
list(
    combinations(
        sorted(
            (
                ("b1",[2,3,4]),
                ("b2",[3,4,77]),
                ("b3", [-1, 67, 0]),
                ("b4",[-2,3,4]),
                ("b5",[3,-42,77]),
                ("b6", [0, 7, -103])
            )
        ),
    2)
)

[(('b1', [2, 3, 4]), ('b2', [3, 4, 77])),
 (('b1', [2, 3, 4]), ('b3', [-1, 67, 0])),
 (('b1', [2, 3, 4]), ('b4', [-2, 3, 4])),
 (('b1', [2, 3, 4]), ('b5', [3, -42, 77])),
 (('b1', [2, 3, 4]), ('b6', [0, 7, -103])),
 (('b2', [3, 4, 77]), ('b3', [-1, 67, 0])),
 (('b2', [3, 4, 77]), ('b4', [-2, 3, 4])),
 (('b2', [3, 4, 77]), ('b5', [3, -42, 77])),
 (('b2', [3, 4, 77]), ('b6', [0, 7, -103])),
 (('b3', [-1, 67, 0]), ('b4', [-2, 3, 4])),
 (('b3', [-1, 67, 0]), ('b5', [3, -42, 77])),
 (('b3', [-1, 67, 0]), ('b6', [0, 7, -103])),
 (('b4', [-2, 3, 4]), ('b5', [3, -42, 77])),
 (('b4', [-2, 3, 4]), ('b6', [0, 7, -103])),
 (('b5', [3, -42, 77]), ('b6', [0, 7, -103]))]

time: 16.5 ms (started: 2024-03-13 02:38:18 +00:00)


In [ ]:
sample_data = [
    '-8O4kt8AIRhM3OUxt-pWLg,_p64KqqRmPwGKhZ-xZwhtg,0.5\n',
    '-A5jntJgFglQ6zwAmOiOMw,cTqIuG-fvlQQL0OWzsFdig,0.5\n',
    '-Jhlh8Scjy669NdtCfKSSg,o5Mofj5KJkYAMs_fhxftpg,0.5\n',
    '-LfTBo0oa_uD454ScEW2XA,J5U-nbhKSnnX7DJGT6QELg,1.0',
    '-LfTBo0oa_uD454ScEW2XA,Lwb6bG1Qu3BbW7FJj5suLw,0.5',
    '-LfTBo0oa_uD454ScEW2XA,aokqWHt8vMf5iwj8KZZ5eg,0.5',
    '-Mm02AeY1PMGg-l-ShMxUg,RWwvuTbnf3DleUSiFbUscg,0.5',
    '-Mm02AeY1PMGg-l-ShMxUg,cKgkSMcPXwWTzPrJRpa2qw,0.5',
    '-N_YCDH4HijYnJ-RisQfHA,W2Vis19kUa7kP6GkSvJ9zQ,0.5',
    '-N_YCDH4HijYnJ-RisQfHA,zm8u9z2KCwEkNiCD59UQ3Q,0.5',
    '-UV2tt3gy57-5a29paA_4w,HUMdEqtlPF70AqyZmiyAhA,0.5',
    '-ePLgQ_afOTW1STxD-2RIA,fBU5QssrXMXPbJWD08o9zg,0.5',
    '-fn65iT4z-nh1Ybfyajxng,9BQGpsIRhs9cRHuGh9xuJw,0.5',
    '-i3dOjumvOw-52aGXU1xDg,6nMYROXu0VX4Ytpdsfi3XA,0.5',
    '-ml57G52t4mQpYRHe54HWQ,Nm9xRoLxn4a_q4XlVAJfSg,0.5',
    '06X34OSshQJPDiyUDHtS0A,or4yipilbCkKIFnF1RP5Ow,0.5',
    '06X34OSshQJPDiyUDHtS0A,rsoxvO7ku3fZdhxAw739bQ,0.5',
    '0BOdyChBbhGjQXRWZtDscQ,uocjMsMLvB9o_0nIRKIR6w,1.0',
    '0FJ22K--OjiiQJZ8VJIpJw,ObJJ9S04UuIIjMHZTbK8Kw,1.0',
    '0JoteSDMdGJSgYzk60vSdQ,NjypbHnZG6HjNQVDVQ_iYA,0.5',
    '0Lq-MK2LwhntBCF7G2t71w,WvcUKsNMERSP_dPkgsq9ZA,0.5',
    '0OOmvIS1Tb5ccYOCvq4Yeg,NKpwKdjEG8nL1rI60aYHmg,0.5',
    '0OOmvIS1Tb5ccYOCvq4Yeg,lnU-G15oaYWwqqbUXj9JEA,0.5',
    '0PenhUhRbw2xd_suA68IjA,7jKDOB9b_nijq0-djwyUTw,0.5',
    '0PenhUhRbw2xd_suA68IjA,_11zD3Vo-oyd21A_t5B3BA,0.5'
]

# Write header and results to a CSV file
header = "business_id_1,business_id_2,similarity\n"
with open("t1.csv", "w") as f:
    f.write(header)
    f.writelines(sample_data)

time: 13.2 ms (started: 2024-03-13 08:44:53 +00:00)


In [ ]:
!cat t1.csv

b,u,s,i,n,e,s,s,_,i,d,_,1,",",b,u,s,i,n,e,s,s,_,i,d,_,2,",",s,i,m,i,l,a,r,i,t,y,"
"
-8O4kt8AIRhM3OUxt-pWLg,_p64KqqRmPwGKhZ-xZwhtg,0.5
-A5jntJgFglQ6zwAmOiOMw,cTqIuG-fvlQQL0OWzsFdig,0.5
-Jhlh8Scjy669NdtCfKSSg,o5Mofj5KJkYAMs_fhxftpg,0.5
-LfTBo0oa_uD454ScEW2XA,J5U-nbhKSnnX7DJGT6QELg,1.0
-LfTBo0oa_uD454ScEW2XA,Lwb6bG1Qu3BbW7FJj5suLw,0.5
-LfTBo0oa_uD454ScEW2XA,aokqWHt8vMf5iwj8KZZ5eg,0.5
-Mm02AeY1PMGg-l-ShMxUg,RWwvuTbnf3DleUSiFbUscg,0.5
-Mm02AeY1PMGg-l-ShMxUg,cKgkSMcPXwWTzPrJRpa2qw,0.5
-N_YCDH4HijYnJ-RisQfHA,W2Vis19kUa7kP6GkSvJ9zQ,0.5
-N_YCDH4HijYnJ-RisQfHA,zm8u9z2KCwEkNiCD59UQ3Q,0.5
-UV2tt3gy57-5a29paA_4w,HUMdEqtlPF70AqyZmiyAhA,0.5
-ePLgQ_afOTW1STxD-2RIA,fBU5QssrXMXPbJWD08o9zg,0.5
-fn65iT4z-nh1Ybfyajxng,9BQGpsIRhs9cRHuGh9xuJw,0.5
-i3dOjumvOw-52aGXU1xDg,6nMYROXu0VX4Ytpdsfi3XA,0.5
-ml57G52t4mQpYRHe54HWQ,Nm9xRoLxn4a_q4XlVAJfSg,0.5
06X34OSshQJPDiyUDHtS0A,or4yipilbCkKIFnF1RP5Ow,0.5
06X34OSshQJPDiyUDHtS0A,rsoxvO7ku3fZdhxAw739bQ,0.5
0BOdyChBbhGjQXRWZtDscQ,uocjMsMLvB9o_0nIRKIR6w,1.0
0FJ22K--OjiiQJZ8

## 2.1

### Version 1

In [6]:
# %%writefile task2_1.py
import sys
import time
import math
import shutil
import statistics
from pyspark import SparkContext, SparkConf
import csv


def prepare_dataset(data, split="train"):
    # Remove the header
    header = data.first()
    data = (
        data.filter(lambda row: row != header)
        .map(lambda row: row.split(","))
        .map(lambda row: (row[0], row[1], row[2]) if split=="train" else (row[0], row[1]))
    )
    return data


def preprocess_train_data(train_data):
    # Group by business_id and collect the corresponding set of users
    bus2user = train_data.map(lambda x: (x[1], x[0])).groupByKey().mapValues(set)
    bus2user = bus2user.collectAsMap()

    # Group by user_id and collect the corresponding set of businesses
    user2bus = train_data.map(lambda x: (x[0], x[1])).groupByKey().mapValues(set)
    user2bus = user2bus.collectAsMap()

    # Group by business_id and collect the corresponding set of users with ratings
    bus2user_rating = train_data.map(lambda x: (x[1], (x[0], float(x[2])))).groupByKey().mapValues(dict)
    bus2user_rating = bus2user_rating.collectAsMap()

    # Calculate average rating for each business
    bus_avg = (
        train_data.map(lambda row: (row[1], float(row[2])))
        .groupByKey()
        .mapValues(lambda ratings: sum(ratings) / len(ratings))
        .collectAsMap()
    )

    # Calculate average rating for each user
    user_avg = (
        train_data.map(lambda row: (row[0], float(row[2])))
        .groupByKey()
        .mapValues(lambda ratings: sum(ratings) / len(ratings))
        .collectAsMap()
    )

    return bus2user, user2bus, bus2user_rating, bus_avg, user_avg


def compute_pearson_similarity(data, item2user_dict, item2user_rating_dict):
    """
    Formala: r = Σᵢ((xᵢ − mean(x))(yᵢ − mean(y))) (√Σᵢ(xᵢ − mean(x))² √Σᵢ(yᵢ − mean(y))²)⁻¹
    """
    # Unpack the data
    item1, item2 = data

    # Find common user to calculate co-rated averages
    common_users = item2user_dict[item1].intersection(item2user_dict[item2])

    # Get ratings of common users for both business
    r1 = [item2user_rating_dict[item1][usr] for usr in common_users]
    r2 = [item2user_rating_dict[item2][usr] for usr in common_users]

    # Center the ratings by subtracting the co-rated average rating
    r1 = [r - statistics.mean(r1) for r in r1]
    r2 = [r - statistics.mean(r2) for r in r2]

    # Compute weight for the item pair
    numer = sum([a * b for a, b in zip(r1, r2)])
    denom = math.sqrt(sum([math.pow(a, 2) for a in r1])) * math.sqrt(sum([math.pow(b, 2) for b in r2]))

    similarity = 0 if denom == 0 else numer/denom

    return similarity


def predict_rating(data, bus2user_dict, user2bus_dict, bus2user_rating_dict, bus_avg, user_avg):
    """Perform Item-based Collaborative filtering on prepared data.
    """
    # Unpack the data
    user, business = data

    # Return avg rating if user or business is not present in the dataset
    if user not in user2bus_dict.keys():
        return 3.5
    if business not in bus2user_dict:
        return user_avg[user]

    # Pearson similarities for rating prediction
    pc = []

    for item in user2bus_dict[user]:
        # Compute pearson similarity for each business pair
        similarity = compute_pearson_similarity((business, item), bus2user_dict, bus2user_rating_dict)

        pc.append((similarity, bus2user_rating_dict[item][user]))

    # Calculate the predicted rating
    top_pc = sorted(pc, key=lambda x: -x[0])[:15]
    x, y = 0, 0
    for p, r in top_pc:
        x += p*r
        y += abs(p)
    predicted_rating = 3.5 if y==0 else x/y

    return predicted_rating


def task2_1(train_file_name, test_file_name, output_file_name):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 2.1: Item-Based Collaborative Filtering")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        start_time = time.time()

        # Read and process the train data
        train_data = spark.textFile(train_file_name)
        train_data = prepare_dataset(train_data, split="train")
        (
            bus2user_dict, user2bus_dict, bus2user_rating_dict, bus_avg, user_avg
        ) = preprocess_train_data(train_data)

        # Read and prepare validation data
        val_data = spark.textFile(test_file_name)
        val_data = prepare_dataset(val_data, split="valid").cache()

        # Predict ratings for validation dataset
        val_data = val_data.map(
            lambda x: [
                x[0],
                x[1],
                predict_rating(x, bus2user_dict, user2bus_dict, bus2user_rating_dict, bus_avg, user_avg)
            ]
        ).cache()

        # header = ["user_id", "business_id", "prediction"]
        # with open(output_file_name, "w", newline="") as f:
        #     writer = csv.writer(f)
        #     writer.writerow(header)
        #     writer.writerows(val_data.collect())

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     if len(sys.argv) != 4:
#         print(
#             "Usage: spark-submit task2_1.py <train_file_name> <test_file_name> <output_file_name>"
#         )
#         sys.exit(1)

#     # Read input parameters
#     train_file_name = sys.argv[1]
#     test_file_name = sys.argv[2]
#     output_file_name = sys.argv[3]

#     task2_1(train_file_name, test_file_name, output_file_name)

task2_1("HW3StudentData/yelp_train.csv", "HW3StudentData/yelp_val.csv", "t2_1.csv")

Duration: 28.488288402557373

time: 37.1 s (started: 2024-03-17 08:36:17 +00:00)


In [ ]:
# !spark-submit task2_1.py HW3StudentData/yelp_train.csv HW3StudentData/yelp_val.csv t2_1.csv

### Version 2

In [54]:
import sys
import time
import math
import shutil
import statistics
from pyspark import SparkContext, SparkConf
import csv


def prepare_dataset(data, split="train"):
    # Remove the header
    header = data.first()
    data = (
        data.filter(lambda row: row != header)
        .map(lambda row: row.split(","))
        .map(lambda row: (row[0], row[1], row[2]) if split=="train" else (row[0], row[1]))
    )
    return data


def preprocess_train_data(train_data):
    # Group by business_id and collect the corresponding set of users
    bus2user = train_data.map(lambda x: (x[1], x[0])).groupByKey().mapValues(set)
    bus2user = bus2user.collectAsMap()

    # Group by user_id and collect the corresponding set of businesses
    user2bus = train_data.map(lambda x: (x[0], x[1])).groupByKey().mapValues(set)
    user2bus = user2bus.collectAsMap()

    # Group by business_id and collect the corresponding set of users with ratings
    bus2user_rating = train_data.map(lambda x: (x[1], (x[0], float(x[2])))).groupByKey().mapValues(dict)
    bus2user_rating = bus2user_rating.collectAsMap()

    # Calculate average rating for each business
    bus_avg = (
        train_data.map(lambda row: (row[1], float(row[2])))
        .groupByKey()
        .mapValues(lambda ratings: sum(ratings) / len(ratings))
        .collectAsMap()
    )

    # Calculate average rating for each user
    user_avg = (
        train_data.map(lambda row: (row[0], float(row[2])))
        .groupByKey()
        .mapValues(lambda ratings: sum(ratings) / len(ratings))
        .collectAsMap()
    )

    return bus2user, user2bus, bus2user_rating, bus_avg, user_avg


def compute_pearson_similarity(data, item2user_dict, item2user_rating_dict):
    """
    Formala: r = Σᵢ((xᵢ − mean(x))(yᵢ − mean(y))) (√Σᵢ(xᵢ − mean(x))² √Σᵢ(yᵢ − mean(y))²)⁻¹
    """
    # Unpack the data
    item1, item2 = data

    # Find common user to calculate co-rated averages
    common_users = item2user_dict[item1].intersection(item2user_dict[item2])

    # Get ratings of common users for both business
    r1, r2 = [], []
    for usr in common_users:
        r1.append(item2user_rating_dict[item1][usr])
        r2.append(item2user_rating_dict[item2][usr])

    # Center the ratings by subtracting the co-rated average rating
    r1 = [r - statistics.mean(r1) for r in r1]
    r2 = [r - statistics.mean(r2) for r in r2]

    # Compute weight for the item pair
    numer = sum([a * b for a, b in zip(r1, r2)])
    denom = math.sqrt(sum([math.pow(a, 2) for a in r1])) * math.sqrt(sum([math.pow(b, 2) for b in r2]))

    similarity = 0 if denom == 0 else numer/denom

    return similarity


def predict_rating(data):
    """Perform Item-based Collaborative filtering on prepared data.
    """
    # Unpack the data
    user, business = data

    # Return avg rating if user or business is not present in the dataset
    if user not in user2bus_dict.keys():
        return 3.5
    if business not in bus2user_dict:
        return user_avg[user]

    # Pearson similarities for rating prediction
    pc = []

    for item in user2bus_dict[user]:
        key = tuple(sorted((item, business)))
        if key not in pearson_similarity.keys():
            # Compute pearson similarity for each business pair
            similarity = compute_pearson_similarity((business, item), bus2user_dict, bus2user_rating_dict)
            pearson_similarity[key] = similarity
        else:
            similarity = pearson_similarity[key]

        pc.append((similarity, bus2user_rating_dict[item][user]))

    # Calculate the predicted rating
    top_pc = sorted(pc, key=lambda x: -x[0])[:15]
    x, y = 0, 0
    for p, r in top_pc:
        x += p*r
        y += abs(p)
    predicted_rating = 3.5 if y==0 else x/y

    return predicted_rating


def task2_1(train_file_name, test_file_name, output_file_name):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 2.1: Item-Based Collaborative Filtering")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        start_time = time.time()

        # Read and process the train data
        train_data = spark.textFile(train_file_name)

        global bus2user_dict, user2bus_dict, bus2user_rating_dict, bus_avg, user_avg
        train_data = prepare_dataset(train_data, split="train")
        (
            bus2user_dict, user2bus_dict, bus2user_rating_dict, bus_avg, user_avg
        ) = preprocess_train_data(train_data)

        # Read and prepare validation data
        val_data = spark.textFile(test_file_name)
        val_data = prepare_dataset(val_data, split="valid").cache()

        global pearson_similarity
        pearson_similarity = {}

        # Predict ratings for validation dataset
        # preds = []
        for pair in val_data.collect():
            pred_r = predict_rating(pair)

        # header = ["user_id", "business_id", "prediction"]
        # with open(output_file_name, "w", newline="") as f:
        #     writer = csv.writer(f)
        #     writer.writerow(header)
        #     writer.writerows(val_data.collect())

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     if len(sys.argv) != 4:
#         print(
#             "Usage: spark-submit task2_1.py <train_file_name> <test_file_name> <output_file_name>"
#         )
#         sys.exit(1)

#     # Read input parameters
#     train_file_name = sys.argv[1]
#     test_file_name = sys.argv[2]
#     output_file_name = sys.argv[3]

#     task2_1(train_file_name, test_file_name, output_file_name)

task2_1("HW3StudentData/yelp_train.csv", "HW3StudentData/yelp_val.csv", "t2_1.csv")

Duration: 330.32587718963623

time: 5min 30s (started: 2024-03-16 08:22:45 +00:00)


### Version 3

### Ref

In [41]:
# @title Reference Code Task 2.1 { vertical-output: true, form-width: "30%" }
%%writefile task2_1_ref.py
from pyspark import SparkContext
import os
import sys
import time

def item_based(bus, user):
    if user not in user_bus_dict.keys():
        return 3.5
    if bus not in bus_user_dict.keys():
        return user_avg_dict[user]

    w_list = []

    for bus1 in user_bus_dict[user]:
        temp = tuple(sorted((bus1, bus)))
        if temp in w_dict.keys():
            w = w_dict[temp]
        else:
            #co-rated
            user_inter = bus_user_dict[bus] & bus_user_dict[bus1]

            if len(user_inter) <= 1:
                w = (5.0 - abs(bus_avg_dict[bus] - bus_avg_dict[bus1])) / 5
            elif len(user_inter) == 2:
                user_inter = list(user_inter)
                w1 = (5.0 - abs(float(bus_user_r_dict[bus][user_inter[0]]) - float(bus_user_r_dict[bus1][user_inter[0]]))) / 5
                w2 = (5.0 - abs(float(bus_user_r_dict[bus][user_inter[1]]) - float(bus_user_r_dict[bus1][user_inter[1]]))) / 5
                w = (w1 + w2) / 2
            else:
                r1 = []
                r2 = []
                for user1 in user_inter:
                    r1.append(float(bus_user_r_dict[bus][user1]))
                    r2.append(float(bus_user_r_dict[bus1][user1]))
                avg1 = sum(r1) / len(r1)
                avg2 = sum(r2) / len(r2)
                temp1 = [x - avg1 for x in r1]
                temp2 = [x - avg2 for x in r2]
                X = (sum([x * y for x,y in zip(temp1, temp2)]))
                Y = ((sum([x ** 2 for x in temp1])**(1/2)) * (sum([x ** 2 for x in temp2])**(1/2)))
                if Y == 0:
                    w = 0
                else:
                    w = X / Y
            w_dict[temp] = w
        w_list.append((w, float(bus_user_r_dict[bus1][user])))
    w_list_can = sorted(w_list, key=lambda x: -x[0])[:15]
    X = 0
    Y = 0
    for w, r in w_list_can:
        X += (w * r)
        Y += abs(w)
    if Y == 0:
        return 3.5
    else:
        return X / Y

if __name__ == '__main__':
    train_path = sys.argv[1]
    val_path = sys.argv[2]
    output_path = sys.argv[3]

    s_t = time.time()

    spark = SparkContext(appName= "task2_1")
    #train data
    lines_train = spark.textFile(train_path)
    first_train = lines_train.first()
    lines_train = lines_train.filter(lambda row: row != first_train).map(lambda row: row.split(",")).map(lambda row: (row[1], row[0], row[2]))

    bus_user_train = lines_train.map(lambda row: (row[0], row[1])).groupByKey().mapValues(set)
    bus_user_dict = {}
    for bus, users in bus_user_train.collect():
        bus_user_dict[bus] = users

    user_bus_train = lines_train.map(lambda row: (row[1], row[0])).groupByKey().mapValues(set)
    user_bus_dict = {}
    for user, bus in user_bus_train.collect():
        user_bus_dict[user] = bus

    #bus_mid = lines_train.map(lambda row: (row[0], float(row[2]))).groupByKey().mapValues(list).map(lambda x: (x[0], sorted(x[1]))).map(lambda x: (x[0], x[1][len(x[1]) // 2]))
    bus_avg = lines_train.map(lambda row: (row[0], float(row[2]))).groupByKey().mapValues(list).map(lambda x: (x[0], sum(x[1]) / len(x[1])))
    bus_avg_dict = {}
    for bus, rating in bus_avg.collect():
        bus_avg_dict[bus] = rating

    #user_mid = lines_train.map(lambda row: (row[1], float(row[2]))).groupByKey().mapValues(list).map(lambda x: (x[0], sorted(x[1]))).map(lambda x: (x[0], x[1][len(x[1]) // 2]))
    user_avg = lines_train.map(lambda row: (row[1], float(row[2]))).groupByKey().mapValues(list).map(lambda x: (x[0], sum(x[1]) / len(x[1])))
    user_avg_dict = {}
    for user, rating in user_avg.collect():
        user_avg_dict[user] = rating

    bus_user_r = lines_train.map(lambda row: (row[0], (row[1], row[2]))).groupByKey().mapValues(set)
    bus_user_r_dict = {}
    for bus, user_r_set in bus_user_r.collect():
        temp = {}
        for user_r in user_r_set:
            temp[user_r[0]] = user_r[1]
        bus_user_r_dict[bus] = temp
    #print(bus_user_r_dict)

    #val data
    lines_val = spark.textFile(val_path)
    first_val = lines_val.first()
    lines_val = lines_val.filter(lambda row: row != first_val).map(lambda row: row.split(",")).map(lambda row: (row[1], row[0]))
    # (bus1, bus2): {simi}
    w_dict = {}

    result_str = "user_id, business_id, prediction\n"
    for row in lines_val.collect():
        prediction = item_based(row[0], row[1])
        result_str += row[1] + "," + row[0] + "," + str(prediction) + "\n"
    with open(output_path, "w") as f:
        f.writelines(result_str)

    e_t = time.time()
    print('Duration: ', e_t - s_t)


    #RMSE: 1.0475857031155809

Writing task2_1_ref.py
time: 14.4 ms (started: 2024-03-15 08:45:21 +00:00)


In [19]:
!spark-submit task2_1_ref.py HW3StudentData/yelp_train.csv HW3StudentData/yelp_val.csv t2-ref.txt --executor-memory 4G --driver-memory 4G

24/03/16 01:52:26 INFO SparkContext: Running Spark version 3.5.1
24/03/16 01:52:26 INFO SparkContext: OS info Linux, 6.1.58+, amd64
24/03/16 01:52:26 INFO SparkContext: Java version 11.0.22
24/03/16 01:52:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 01:52:26 INFO ResourceUtils: ==============================================================
24/03/16 01:52:26 INFO ResourceUtils: No custom resources configured for spark.driver.
24/03/16 01:52:26 INFO ResourceUtils: ==============================================================
24/03/16 01:52:26 INFO SparkContext: Submitted application: task2_1
24/03/16 01:52:26 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amou

### Test

In [65]:
df = pd.read_csv("HW3StudentData/yelp_val.csv")
# df.loc[df["user_id"]=="wf1GqnKQuvH-V3QN80UOOQ"]
df

,user_id,business_id,stars
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg,5.0
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ,5.0
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA,4.0
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA,5.0
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw,3.0
...,...,...,...
142039,pA9NXgASl86RImkdBtydrA,q6-SF8zHFU1AWO70k92o1Q,2.0
142040,_eUb7UGsUoSfi9n2ieF5ow,hgWMxKhrnOUd3m5nOUBIkA,4.0
142041,cEJGXB63KhROA-XmE_jgXw,0ldxjei8v4q95fApIei3Lg,5.0
142042,Z4-V0hc51oxUdULWJOufeg,j29tuUdrfaxmGjwxHdHZPA,3.0


time: 317 ms (started: 2024-03-15 23:53:20 +00:00)


## 2.2

In [20]:
# %%writefile task2_2.py
import csv
import json
import sys
import time

import numpy as np
from pyspark import SparkConf, SparkContext
from xgboost import XGBRegressor


def save_data(data, output_file_name):
    header = ["user_id", "business_id", "prediction"]
    with open(output_file_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)


def read_csv_spark(path, sc):
    rdd = sc.textFile(path)
    header = rdd.first()
    rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
    return rdd


def read_json_spark(path, sc):
    return sc.textFile(path).map(lambda row: json.loads(row))


def process_reviews(review_rdd):
    review_rdd = (
        review_rdd.map(
            lambda row: (row["business_id"], (float(row["useful"]), float(row["funny"]), float(row["cool"])))
        )
        .groupByKey()
        .mapValues(lambda x: tuple(sum(col) / len(col) for col in zip(*x)))
        .cache()
    )
    return review_rdd.collectAsMap()


def process_user(usr_rdd):
    usr_rdd = usr_rdd.map(
        lambda row: (row["user_id"], (float(row["average_stars"]), float(row["review_count"]), float(row["fans"])))
    ).cache()
    return usr_rdd.collectAsMap()


def process_bus(bus_rdd):
    bus_rdd = bus_rdd.map(lambda row: (row["business_id"], (float(row["stars"]), float(row["review_count"])))).cache()
    return bus_rdd.collectAsMap()


def process_train_data(row, review_dict, usr_dict, bus_dict):
    if len(row)==3:
        usr, bus, rating = row
    else:
        usr, bus = row
        rating = 0

    useful, funny, cool = review_dict.get(bus, (None, None, None))
    usr_avg_star, usr_review_cnt, usr_fans = usr_dict.get(usr, (None, None, None))
    bus_avg_star, bus_review_cnt = bus_dict.get(bus, (None, None))

    return ([useful, funny, cool, usr_avg_star, usr_review_cnt, usr_fans, bus_avg_star, bus_review_cnt], rating)


def task2_2(folder_path, test_file_name, output_file_name):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 2.2: : Model-based recommendation system")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        start_time = time.time()

        # Read and process the train data
        train_rdd = read_csv_spark(folder_path + "/yelp_train.csv", spark)

        review_rdd = read_json_spark(folder_path + "/review_train.json", spark)
        review_rdd = process_reviews(review_rdd)

        usr_rdd = read_json_spark(folder_path + "/user.json", spark)
        usr_rdd = process_user(usr_rdd)

        bus_rdd = read_json_spark(folder_path + "/business.json", spark)
        bus_rdd = process_bus(bus_rdd)

        # Read and process validation dataset
        val_rdd = read_csv_spark(test_file_name, spark).cache()

        # Train X and Y
        train_rdd = train_rdd.map(lambda x: process_train_data(x, review_rdd, usr_rdd, bus_rdd))

        # Valid x and Y
        val_processed = val_rdd.map(lambda x: process_train_data(x, review_rdd, usr_rdd, bus_rdd))

        # Extract X_train and Y_train
        X_train = train_rdd.map(lambda x: x[0]).cache()
        X_train = np.array(X_train.collect(), dtype="float32")
        Y_train = train_rdd.map(lambda x: x[1]).cache()
        Y_train = np.array(Y_train.collect(), dtype="float32")

        # Extract X_train and Y_train
        X_val = val_processed.map(lambda x: x[0]).cache()
        X_val = np.array(X_val.collect(), dtype="float32")
        # Y_val = val_processed.map(lambda x: x[1]).cache()
        # Y_val = np.array(Y_val.collect(), dtype='float32')

        xgb = XGBRegressor(
            colsample_bytree=0.5,
            subsample=0.8,
            learning_rate=0.02,
            max_depth=17,
            random_state=47,
            min_child_weight=101,
            n_estimators=40,
        )
        xgb.fit(X_train, Y_train)
        Y_pred = xgb.predict(X_val)

        pred_data = []
        for i, row in enumerate(val_rdd.collect()):
            pred_data.append([row[0], row[1], Y_pred[i]])

        save_data(pred_data, output_file_name)

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Usage: spark-submit task2_1.py <folder_path> <test_file_name> <output_file_name>")
        sys.exit(1)

    # Read input parameters
    folder_path = sys.argv[1]
    test_file_name = sys.argv[2]
    output_file_name = sys.argv[3]

    task2_2(folder_path, test_file_name, output_file_name)

# task2_2("HW3StudentData", "HW3StudentData/yelp_val.csv", "t2_2.csv")


Duration: 89.79863214492798

time: 1min 31s (started: 2024-03-17 02:54:04 +00:00)


In [ ]:
!spark-submit task2_2.py HW3StudentData HW3StudentData/yelp_val.csv t2_2-dryrun.csv

## 2.3

# THE END